In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
import pymysql
from datetime import datetime, timedelta

# Function to convert time strings to datetime format
def convert_to_datetime(time_str, reference_date):
    try:
        dt = datetime.strptime(time_str, '%H:%M').replace(year=reference_date.year, month=reference_date.month, day=reference_date.day)
        return dt
    except ValueError:
        return None

#Initialize the webdriver
driver = webdriver.Chrome()
driver.maximize_window()

#Open Redbus website
driver.get("https://www.redbus.in/")
time.sleep(10)

#Click view all in Government bus section
click_view_all = driver.find_element(By.XPATH, '//*[@id="homeV2-root"]/div[3]/div[1]/div[2]/a')
click_view_all.click()
time.sleep(3)

#Switching to new tab
driver.switch_to.window(driver.window_handles[1])

#Scrolling down to the bottom of the page
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

#Selecting bus (KSTRC)

try:
    bus_corp = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="root"]/div/article[2]/div/div/ul[3]/li[4]/a'))
    )
    
    #Scroll the element to view
    driver.execute_script("arguments[0].scrollIntoView(true);", bus_corp)
    time.sleep(5)
    #Click the KSTRC link
    bus_corp.click()
    time.sleep(5)

except ElementClickInterceptedException:
    print("There is an error")
    bus_corp.click()
    time.sleep(5)
    
#Initializing a list to store routes
routes = []

#Loop through each pages manually to scrape data
while True:
    input("Please click the next page number button manually, wait for the page to load, then press Enter to confirm...")

    # Scrape route names and their links
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    for route_element in route_elements:
        route = route_element.text  # Get the route name
        route_link = route_element.get_attribute('href')  # Get the route link
        routes.append((route, route_link))
    
    # Check if there are more pages to click
    more_pages = input("Are there more pages to click? (yes/no): ")
    if more_pages.lower() != 'yes':
        break

#Printing scraped routes
print("Scraped routes and links")
for route, link in routes:
    print(f"Route:{route}, Link:{link}")

#Using scraped routes and links to get bus details
bus_details=[]

for route, route_link in routes:
    driver.get(route_link)
    time.sleep(5)

    #Clicking on view buses button if present
    try:
        view_buses_button = driver.find_element(By.CSS_SELECTOR, 'div.button')
        view_buses_button.click()
        time.sleep(5)
    except NoSuchElementException:
        pass
        
    # Scroll down to the bottom of the page
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    try:
        bus_elements = driver.find_elements(By.CSS_SELECTOR, "div.bus-item")
    except NoSuchElementException:
        print("No bus elements found")
        continue

    for bus in bus_elements:
        try:
            bus_name = bus.find_element(By.CSS_SELECTOR, "div.travels.lh-24.f-bold.d-color").text
        except NoSuchElementException:
            bus_name = "N/A"

        try:
            bus_type = bus.find_element(By.CSS_SELECTOR, 'div.bus-type.f-12.m-top-16.l-color.evBus').text
        except NoSuchElementException:
            bus_type = "N/A"

        try:
            departing_time = bus.find_element(By.CSS_SELECTOR, 'div.dp-time.f-19.d-color.f-bold').text
            departing_time_dt = convert_to_datetime(departing_time, datetime.now())
        except NoSuchElementException:
            departing_time_dt = None

        try:
            duration = bus.find_element(By.CSS_SELECTOR, 'div.dur l-color lh-24').text
        except NoSuchElementException:
            duration = "N/A"

        try:
            arrival_time = bus.find_element(By.CSS_SELECTOR, 'div.bp-time.f-19.d-color.disp-Inline').text
            arrival_time_at = convert_to_datetime(arrival_time, datetime.now())
            if arrival_time_at and departing_time_dt and arrival_time_at < departing_time_dt:
                arrival_time_at += timedelta(days=1)
        except NoSuchElementException:
            arrival_time_at = None

        try:
            rating = bus.find_element(By.CSS_SELECTOR, 'div.rating-sec.lh-24').text
            rating = float(rating) if rating != "N/A" else 0.0
        except NoSuchElementException:
            rating = 0.0

        try:
            price = bus.find_element(By.CSS_SELECTOR, "span.f-19.f-bold").text
            price = float(price.replace('INR', '').replace('Starts from', '').replace('600 500\nredDeal applied', '').replace(',', '').strip()) if price != "N/A" else None
        except NoSuchElementException:
            price = None

        try:
            # Try the first selector for seats available
            try:
                seats_available = bus.find_element(By.CSS_SELECTOR, "div.seat-left.m-top-16").text
            except NoSuchElementException:
                # If the first selector fails, try the second one
                seats_available = bus.find_element(By.CSS_SELECTOR, "div.seat-left.m-top-30").text  # Replace with the actual second selector

            seats_available = int(seats_available.split()[0]) if seats_available != "N/A" else 0
        except NoSuchElementException:
            seats_available = 0

        bus_details.append((route, route_link, bus_name, bus_type, departing_time_dt, duration, arrival_time_at, rating, price, seats_available))

#To print the scraped data
print("Scraped Bus Details")
for detail in bus_details:
    print(detail)

#Connect to MySQL database
conn = pymysql.connect(
    host = '127.0.0.1',
    user='root',
    passwd='root',
    db='redbus_data'
)
cursor = conn.cursor()

#Creating a bus route table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS bus_routes (
        id INT AUTO_INCREMENT PRIMARY KEY,
        route_name TEXT,
        route_link TEXT,
        busname TEXT,
        bustype TEXT,
        departing_time DATETIME,
        duration TEXT,
        reaching_time DATETIME,
        star_rating FLOAT,
        price DECIMAL(10, 2),
        seats_available INT
    )
''')

#Insert scraped date into the table
for detail in bus_details:
    cursor.execute('''
        INSERT INTO bus_routes (
            route_name, route_link, busname, bustype, departing_time,
            duration, reaching_time, star_rating, price, seats_available
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ''', detail)

# Commit the transaction and close the connection
conn.commit()
conn.close()

print("Data has been successfully saved to the database.")

# Close the WebDriver
driver.quit()